In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator

0it [00:00, ?it/s]
/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [115]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [7]:
key = pd.read_csv('./key_trunc.csv')

In [8]:
sub = pd.read_csv("./sample_submission_1.csv")

In [76]:
train_melt = pd.read_csv("train_feats.csv")

In [234]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

KeyboardInterrupt: 

In [125]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days, min_periods=7)[-60:]    

In [126]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days, min_periods=7)[-60:]    

In [86]:
def get_mean_diff(row, last_n_days):
    return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [87]:
def get_last_day_visits(row):
    return row[1:]

In [128]:
def get_long_stats(func_type, df, df_melt, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.progress_apply(get_median, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.progress_apply(get_mean, last_n_days = last_n_days, axis = 1)
        
#     elif func_type == "mean_diff":
#         rolling_stats = df.iloc[:, -61:].progress_apply(get_mean_diff, last_n_days = last_n_days, axis = 1)
        
#     elif func_type == "last_day_visits":        
#         rolling_stats = df.iloc[:, -61:].progress_apply(get_last_day_visits, axis = 1)

    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [129]:
def generate_train_melt(train):
    
#     train_melt = pd.read_csv("train_feats.csv")
    
#     Commented because these features are already generated
    train_melt = pd.melt(pd.concat([train.Page, train.iloc[:, -60:]], axis = 1), id_vars=['Page'], var_name="date", value_name="visits")

#     train_melt = get_long_stats("mean", train, train_melt, 7)
    train_melt = get_long_stats("mean", train, train_melt, 30)
    train_melt = get_long_stats("mean", train, train_melt, 60)

#     train_melt = get_long_stats("median", train, train_melt, 7)
    train_melt = get_long_stats("median", train, train_melt, 30)
    train_melt = get_long_stats("median", train, train_melt, 60)

#     train_melt = get_long_stats("mean_diff", train, train_melt, 7)
#     train_melt = get_long_stats("last_day_visits", train, train_melt)
    
    train_melt["date_num"] = train_melt.progress_apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.progress_apply(get_weekday, axis = 1)
    
    # Columns i am not using
    # train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)
    # train_melt["language"] = train_melt.progress_apply(get_language, axis = 1)
    
    return train_melt 

train_melt = generate_train_melt(train)
train_melt.to_csv("train_feats.csv", index = False)





  0%|          | 0/145063 [00:00<?, ?it/s]



  0%|          | 1/145063 [00:11<472:59:02, 11.74s/it]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=7,window=30,center=False).mean()
  




  0%|          | 2/145063 [00:18<418:29:05, 10.39s/it]



  0%|          | 126/145063 [00:19<292:41:55,  7.27s/it]



  0%|          | 264/145063 [00:19<204:42:10,  5.09s/it]



  0%|          | 405/145063 [00:19<143:09:39,  3.56s/it]



  0%|          | 551/145063 [00:19<100:07:11,  2.49s/it]



  0%|          | 708/145063 [00:19<70:00:55,  1.75s/it] 



  1%|          | 854/145063 [00:19<48:58:10,  1.22s/it]



  1%|          | 1012/145063 [00:19<34:14:55,  1.17it/s]



  1%|          | 1173/145063 [00:19<23:57:17,  1.67it/s]



  1%|          | 1325/145063 [00:19<16:45:30,  2.38it/s]



  1%|          | 1475/145063 [00:19<11:43:36,  3.40it/s]



 

 11%|█▏        | 16470/145063 [00:32<01:43, 1242.12it/s]



 11%|█▏        | 16627/145063 [00:32<01:36, 1324.34it/s]



 12%|█▏        | 16784/145063 [00:32<01:33, 1378.35it/s]



 12%|█▏        | 16940/145063 [00:32<01:30, 1409.14it/s]



 12%|█▏        | 17094/145063 [00:32<01:29, 1435.81it/s]



 12%|█▏        | 17247/145063 [00:32<01:27, 1461.56it/s]



 12%|█▏        | 17400/145063 [00:32<01:26, 1473.15it/s]



 12%|█▏        | 17553/145063 [00:32<01:25, 1489.68it/s]



 12%|█▏        | 17709/145063 [00:33<01:24, 1509.86it/s]



 12%|█▏        | 17873/145063 [00:33<01:22, 1545.55it/s]



 12%|█▏        | 18030/145063 [00:33<01:23, 1516.39it/s]



 13%|█▎        | 18184/145063 [00:33<01:24, 1493.96it/s]



 13%|█▎        | 18335/145063 [00:33<01:25, 1489.21it/s]



 13%|█▎        | 18496/145063 [00:33<01:23, 1521.37it/s]



 13%|█▎        | 18652/145063 [00:33<01:22, 1531.63it/s]



 13%|█▎        | 18809/145063 [00:33<01:21, 1540.97it/s]



 13%|█▎        | 18968/145063 [00:33<01:

 23%|██▎       | 33817/145063 [00:43<01:15, 1473.78it/s]



 23%|██▎       | 33965/145063 [00:44<01:16, 1457.15it/s]



 24%|██▎       | 34111/145063 [00:44<01:16, 1455.03it/s]



 24%|██▎       | 34257/145063 [00:44<01:16, 1448.42it/s]



 24%|██▎       | 34403/145063 [00:44<01:16, 1450.70it/s]



 24%|██▍       | 34549/145063 [00:44<01:17, 1428.24it/s]



 24%|██▍       | 34698/145063 [00:44<01:16, 1445.79it/s]



 24%|██▍       | 34854/145063 [00:44<01:14, 1477.73it/s]



 24%|██▍       | 35003/145063 [00:45<04:40, 392.09it/s] 



 24%|██▍       | 35150/145063 [00:45<03:38, 502.53it/s]



 24%|██▍       | 35303/145063 [00:45<02:54, 629.03it/s]



 24%|██▍       | 35457/145063 [00:45<02:23, 764.04it/s]



 25%|██▍       | 35600/145063 [00:46<02:03, 887.97it/s]



 25%|██▍       | 35742/145063 [00:46<01:49, 999.17it/s]



 25%|██▍       | 35890/145063 [00:46<01:38, 1106.84it/s]



 25%|██▍       | 36034/145063 [00:46<01:31, 1188.38it/s]



 25%|██▍       | 36192/145063 [00:46<01:24, 1

 35%|███▍      | 50678/145063 [00:56<01:03, 1491.38it/s]



 35%|███▌      | 50828/145063 [00:56<01:03, 1474.23it/s]



 35%|███▌      | 50976/145063 [00:56<01:04, 1466.09it/s]



 35%|███▌      | 51127/145063 [00:56<01:03, 1476.47it/s]



 35%|███▌      | 51275/145063 [00:56<01:04, 1457.62it/s]



 35%|███▌      | 51438/145063 [00:56<01:02, 1503.84it/s]



 36%|███▌      | 51593/145063 [00:56<01:01, 1516.27it/s]



 36%|███▌      | 51745/145063 [00:57<01:01, 1506.48it/s]



 36%|███▌      | 51900/145063 [00:57<01:01, 1518.32it/s]



 36%|███▌      | 52053/145063 [00:57<01:01, 1516.64it/s]



 36%|███▌      | 52205/145063 [00:57<01:02, 1478.56it/s]



 36%|███▌      | 52355/145063 [00:57<01:02, 1482.50it/s]



 36%|███▌      | 52504/145063 [00:57<01:02, 1476.35it/s]



 36%|███▋      | 52652/145063 [00:57<01:02, 1470.34it/s]



 36%|███▋      | 52802/145063 [00:57<01:02, 1479.00it/s]



 37%|███▋      | 52950/145063 [00:57<01:02, 1465.02it/s]



 37%|███▋      | 53105/145063 [00:57<01:

 47%|████▋     | 67890/145063 [01:09<00:50, 1523.73it/s]



 47%|████▋     | 68049/145063 [01:09<00:49, 1541.70it/s]



 47%|████▋     | 68204/145063 [01:09<00:49, 1538.92it/s]



 47%|████▋     | 68358/145063 [01:09<00:51, 1493.47it/s]



 47%|████▋     | 68508/145063 [01:09<00:52, 1456.56it/s]



 47%|████▋     | 68655/145063 [01:09<00:52, 1449.31it/s]



 47%|████▋     | 68803/145063 [01:09<00:52, 1458.00it/s]



 48%|████▊     | 68950/145063 [01:09<00:52, 1459.92it/s]



 48%|████▊     | 69108/145063 [01:10<00:50, 1492.42it/s]



 48%|████▊     | 69258/145063 [01:10<00:51, 1483.66it/s]



 48%|████▊     | 69413/145063 [01:10<00:50, 1499.49it/s]



 48%|████▊     | 69569/145063 [01:10<00:49, 1516.22it/s]



 48%|████▊     | 69722/145063 [01:10<00:49, 1520.18it/s]



 48%|████▊     | 69884/145063 [01:10<00:48, 1546.85it/s]



 48%|████▊     | 70039/145063 [01:10<00:48, 1538.06it/s]



 48%|████▊     | 70193/145063 [01:10<00:49, 1527.25it/s]



 49%|████▊     | 70357/145063 [01:10<00:

...

 42%|████▏     | 3627523/8703780 [01:31<02:01, 41822.16it/s]



 42%|████▏     | 3631724/8703780 [01:31<02:02, 41239.86it/s]



 42%|████▏     | 3636345/8703780 [01:32<01:58, 42612.55it/s]



 42%|████▏     | 3640629/8703780 [01:32<02:01, 41647.84it/s]



 42%|████▏     | 3644814/8703780 [01:32<02:02, 41163.43it/s]



 42%|████▏     | 3648946/8703780 [01:32<02:03, 40781.39it/s]



 42%|████▏     | 3653036/8703780 [01:32<02:04, 40476.76it/s]



 42%|████▏     | 3657324/8703780 [01:32<02:02, 41168.13it/s]



 42%|████▏     | 3662030/8703780 [01:32<01:57, 42771.13it/s]



 42%|████▏     | 3666330/8703780 [01:32<01:59, 42161.34it/s]



 42%|████▏     | 3670564/8703780 [01:32<02:00, 41678.98it/s]



 42%|████▏     | 3674746/8703780 [01:32<02:02, 41027.65it/s]



 42%|████▏     | 3678861/8703780 [01:33<02:03, 40690.07it/s]



 42%|████▏     | 3683117/8703780 [01:33<02:01, 41230.69it/s]



 42%|████▏     | 3687772/8703780 [01:33<01:57, 42693.59it/s]



 42%|████▏     | 3692061/8703780 [01:33<

 47%|████▋     | 4081245/8703780 [01:42<01:51, 41560.61it/s]



 47%|████▋     | 4085405/8703780 [01:42<01:53, 40639.03it/s]



 47%|████▋     | 4089708/8703780 [01:43<01:51, 41325.55it/s]



 47%|████▋     | 4094050/8703780 [01:43<01:49, 41930.92it/s]



 47%|████▋     | 4098251/8703780 [01:43<01:52, 41085.80it/s]



 47%|████▋     | 4102369/8703780 [01:43<01:52, 40900.49it/s]



 47%|████▋     | 4106466/8703780 [01:43<01:53, 40536.63it/s]



 47%|████▋     | 4110525/8703780 [01:43<01:56, 39422.66it/s]



 47%|████▋     | 4114691/8703780 [01:43<01:54, 40066.46it/s]



 47%|████▋     | 4118732/8703780 [01:43<01:54, 40168.55it/s]



 47%|████▋     | 4122976/8703780 [01:43<01:52, 40823.21it/s]



 47%|████▋     | 4127259/8703780 [01:44<01:50, 41402.82it/s]



 47%|████▋     | 4131407/8703780 [01:44<01:56, 39158.45it/s]



 48%|████▊     | 4135354/8703780 [01:44<01:57, 38768.75it/s]



 48%|████▊     | 4139305/8703780 [01:44<01:57, 38986.57it/s]



 48%|████▊     | 4143324/8703780 [01:44<

 52%|█████▏    | 4537853/8703780 [01:54<01:38, 42220.30it/s]



 52%|█████▏    | 4542609/8703780 [01:54<01:35, 43689.87it/s]



 52%|█████▏    | 4547004/8703780 [01:54<01:38, 42224.66it/s]



 52%|█████▏    | 4551255/8703780 [01:54<01:39, 41595.47it/s]



 52%|█████▏    | 4555437/8703780 [01:54<01:40, 41254.45it/s]



 52%|█████▏    | 4559584/8703780 [01:54<01:40, 41316.15it/s]



 52%|█████▏    | 4564359/8703780 [01:54<01:36, 43056.02it/s]



 52%|█████▏    | 4568692/8703780 [01:54<01:38, 41954.45it/s]



 53%|█████▎    | 4572913/8703780 [01:54<01:40, 41119.70it/s]



 53%|█████▎    | 4577047/8703780 [01:54<01:41, 40630.56it/s]



 53%|█████▎    | 4581573/8703780 [01:55<01:38, 41914.46it/s]



 53%|█████▎    | 4585786/8703780 [01:55<01:40, 41105.63it/s]



 53%|█████▎    | 4589916/8703780 [01:55<01:41, 40484.03it/s]



 53%|█████▎    | 4593980/8703780 [01:55<01:42, 40265.39it/s]



 53%|█████▎    | 4598018/8703780 [01:55<01:42, 40102.43it/s]



 53%|█████▎    | 4602036/8703780 [01:55<

KeyboardInterrupt: 




 42%|████▏     | 61091/145063 [34:36<47:34, 29.42it/s]  

In [7]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [8]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [94]:
def get_month(row):
    return row.date.split("-")[1]

In [95]:
def get_date(row):
    return row.date.split("-")[2]

In [96]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [22]:
def get_median_test(row, last_index, last_n_days):
    
    start_loc = max(min(last_index - last_n_days, 61), 0)
    end_loc = min(last_index, 61)

    if start_loc < end_loc:
        return np.median(row.iloc[ start_loc : end_loc].tolist())
    else:
        return np.median(row.iloc[61 - last_n_days : 61].tolist())


In [23]:
def get_mean_test(row, last_index, last_n_days):
    
    start_loc = max(min(last_index - last_n_days, 61), 0)
    end_loc = min(last_index, 61)
    
    if start_loc < end_loc:
        return np.mean(row.iloc[start_loc : end_loc].tolist())
    else:
        return np.mean(row.iloc[61 - last_n_days : 61].tolist())

In [12]:
def get_mean_diff_test(row, last_index):
    
    start_loc = min(last_index - 7, 61)
    end_loc = min(last_index, 61)
    
    if start_loc < end_loc:
        return np.mean([int(x) for x in np.diff( row.iloc[start_loc : end_loc] ) ], dtype=None)
    else:
        return np.nan

In [13]:
def get_last_day_visits_test(row, last_index):
    
    return row.iloc[last_index - 1]

In [18]:
# def get_test_feats(row, last_index):
    
#     subset_row = row.iloc[last_index - 60 : last_index].tolist()
    
#     median_7 = np.median(subset_row[-7:])
#     median_30 = np.median(subset_row[-30:])
#     median_60 = np.median(subset_row[-60:])

#     mean_7 = np.mean(subset_row[-7:])
#     mean_30 = np.mean(subset_row[-30:])
#     mean_60 = np.mean(subset_row[-60:])
    
#     return pd.Series([row.Page, median_7, median_30, median_60, mean_7, mean_30, mean_60])

In [14]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [38]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.05
    params['max_depth'] = 7
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0

    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)
    
    return bst

In [79]:
label_train_melt = train_melt["visits"]
# label_train_melt = [0] + np.diff(train_melt["visits"]).tolist()

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

train_melt.drop("last_day_visits_None", axis = 1, inplace = True)
train_melt.drop("mean_diff_7", axis = 1, inplace = True)

train_melt.drop("mean_7", axis = 1, inplace = True)
train_melt.drop("median_7", axis = 1, inplace = True)

train_melt.date_num = pd.to_numeric(train_melt.date_num)

In [80]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)

x_train = x_train.fillna(0)
x_valid = x_valid.fillna(0)

reg.fit(x_train, label_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [81]:
pred = reg.predict(x_valid)

In [82]:
y_true = np.array(label_valid)
y_pred = np.array(pred)

denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
diff = np.abs(y_true - y_pred) / denominator
diff[denominator == 0] = 0.0

print np.nanmean(diff)

129.897826373


In [53]:
x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
bst = run_xgb(x_train, label_train, x_valid, label_valid)

# bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:113044	validation-rmse:121182	train-smape:176.038	validation-smape:174.376
Multiple eval metrics have been passed: 'validation-smape' will be used for early stopping.

Will train until validation-smape hasn't improved in 10 rounds.
[10]	train-rmse:112826	validation-rmse:121147	train-smape:162.418	validation-smape:163.085
[20]	train-rmse:112701	validation-rmse:121139	train-smape:161.8	validation-smape:162.643
Stopping. Best iteration:
[16]	train-rmse:112741	validation-rmse:121141	train-smape:161.605	validation-smape:162.391



In [41]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

     feature    fscore
0    mean_60  0.049815
1  median_60  0.065298
2    weekday  0.139010
3   date_num  0.217772
4    mean_30  0.235947
5  median_30  0.292158


In [16]:
def get_preds(test_df):
    
    d_test = xgb.DMatrix(test_df)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [132]:
def get_overall_df():
    
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = 0

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [18]:
def get_test_df(overall_df):
    
    test_df = pd.DataFrame()

    pred_index = overall_df.columns.tolist().index('2017-01-01')
    
    for i in range(pred_index, len(overall_df.columns)):

        last_date = overall_df.columns.tolist()[i]
        print last_date
        last_index = i

        test_feats = pd.DataFrame()
        
        test_feats["Page"] = overall_df.Page
        test_feats["date"] = last_date
        
        test_feats["mean_7"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 7)
        test_feats["mean_30"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 30)
        test_feats["mean_60"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 60)

        test_feats["median_7"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 7)
        test_feats["median_30"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 30)
        test_feats["median_60"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 60)

#         test_feats["mean_diff_7"] = overall_df.apply(get_mean_diff_test, axis = 1, last_index = last_index)
#         test_feats["last_day_visits_None"] = overall_df.apply(get_last_day_visits_test, axis = 1, last_index = last_index)

#         test_feats["month"] = last_date.split("-")[1]
        test_feats["date_num"] = pd.to_numeric(last_date.split("-")[2])
        test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()
                                
        test_df = pd.concat([test_df, test_feats])
        
    return test_df

In [133]:
overall_df = get_overall_df()
test_df = get_test_df(overall_df)
test_df.to_csv("test_feats.csv", index = False)

In [42]:
# test_df = pd.read_csv("test_feats.csv")
# prediction = get_preds(test_df[["mean_7","mean_30","mean_60","median_7","median_30","median_60","date_num", "weekday"]])
prediction = get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday"]])

In [43]:
# pred_index = overall_df.columns.tolist().index("2017-01-01")

# pred_df = overall_df.iloc[:, pred_index:]
# pred_df["Page"] = overall_df.Page

# pred_df_melt = pd.melt(pred_df, id_vars=['Page'], var_name="date", value_name="Visits")        

test_df["Visits"] = prediction
# pred_df = pd.concat([test_df[["Page", "date"]], prediction], axis = 1)
pred_df = test_df[["Page", "date", "Visits"]]

pred_id_visits = pd.merge(pred_df, key, left_on=["Page", "date"], right_on=["trunc_page", "date"], how="inner")[["Id", "Visits"]]

pred_id_visits.to_csv("pred_xgb_4.csv", index = False)

